In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time

In [2]:
#using chromedriver
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

### NASA Mars News

In [3]:
# Visit mars website
url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(3)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [4]:
#Scrape data and view it
news = soup.find("li", class_="slide")
print(news)

<li class="slide"><div class="image_and_description_container"><a href="/news/8801/hear-audio-from-nasas-perseverance-as-it-travels-through-deep-space/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="Location of microphone on Perseverance (Illustration): " src="/system/news_items/list_view_images/8801_20201118_mars2020-320x240.jpg"/></div><div class="bottom_gradient"><div><h3>Hear Audio From NASA's Perseverance As It Travels Through Deep Space</h3></div></div></a><div class="list_text"><div class="list_date">November 18, 2020</div><div class="content_title"><a href="/news/8801/hear-audio-from-nasas-perseverance-as-it-travels-through-deep-space/" target="_self">Hear A

In [5]:
#Get the title and description of the latest news
news_title = news.find("div", class_="content_title").text
news_p = news.find("div", class_="article_teaser_body").text
print(news_title)
print("----------")
print(news_p)

Hear Audio From NASA's Perseverance As It Travels Through Deep Space
----------
The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.


### JPL Mars Space Images - Features Image

In [6]:
# Visit the image url
image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image)

time.sleep(3)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [7]:
#find article for featured image and print
img_info = soup.find("li",class_="slide").a["data-fancybox-href"]
base_url = "https://www.jpl.nasa.gov"
featured_image_url = base_url + img_info
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24247_hires.jpg


### Mars Facts

In [8]:
# Visit the facts url
spacefacts = 'https://space-facts.com/mars/'
df = pd.read_html(spacefacts)
df

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [9]:
mars_df = df[0]
mars_df = mars_df.rename({0: "", 1: "Mars Statistics"}, axis =1).set_index("")
mars_df

,Mars Statistics
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
mars_df.to_html("mars_facts.html")

### Mars Hemispheres

In [11]:
# Visit the hemisphers url
hem = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hem)

time.sleep(3)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser") 

In [12]:
#Find the image urls
img_links= soup.find_all("div", class_="item")
print(img_links)

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>, <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapar

In [13]:
#Create loop to get all links
hemisphere_image_urls = []
for img in img_links:
    # Get title
    title = img.find("h3").text
    #Click onto image url to get better image
    extra = img.find("a")["href"]
    base = "https://astrogeology.usgs.gov"
    get_url = base + extra
    response = requests.get(get_url)
    #Soup it up
    soup = bs(response.text, "html.parser") 
    #Get full res image
    full_res_img = soup.find("img", class_="wide-image")["src"]
    img_url = base + full_res_img
    hemisphere_image_urls.append({"title": title, "img_url": img_url})
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]